In [1]:
import numpy as np                   # advanced math library
import matplotlib.pyplot as plt 
import seaborn as sns     # MATLAB like plotting routines
import random                        # for generating random numbers
#
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
#
import os
#
import pandas as pd
import tensorflow as tf
#
from tensorflow.keras.models import Model
#
from tensorflow.keras.layers import Dense, Flatten, Input, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
#
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

from PIL import Image as im
#
import scipy.sparse
#
print(tf.__version__)

from keras.datasets import mnist     # MNIST dataset is included in Keras
from keras.models import Sequential  # Model type to be used


from keras.layers.core import Dense, Dropout, Activation # Types of layers to be used in our model
from keras.utils import np_utils      

2.10.0


In [2]:
b = np.load('movie_recommend.npz')
print(b.files)

['format', 'shape', 'data', 'row', 'col']


In [3]:
print(b['data']) #data = 1000209

[5 5 5 ... 4 4 5]


In [4]:
# Carga el archivo .npz
archive_title_npz = np.load('movie_titles.npz')
archive_movie_npz= np.load('movie_recommend.npz')

# Claves disponibles en el archivo de títulos
print("Claves disponibles en el archivo de títulos:", archive_title_npz.files)

# claves disponibles en el archivo de recomendaciones de películas
print("Claves disponibles en el archivo de recomendaciones de películas:", archive_movie_npz.files)

# Obtiene el array de títulos
title_array = archive_title_npz['titles']

Claves disponibles en el archivo de títulos: ['titles']
Claves disponibles en el archivo de recomendaciones de películas: ['format', 'shape', 'data', 'row', 'col']


In [5]:
len(title_array)

3706

In [6]:
sparse_matrix = scipy.sparse.load_npz('movie_recommend.npz')

In [7]:
sparse_matrix #6040 users and 3706 movies

<6040x3706 sparse matrix of type '<class 'numpy.int8'>'
	with 1000209 stored elements in COOrdinate format>

In [8]:
sparse_matrix = sparse_matrix.toarray()

In [9]:
def recommend_system(sdata):
    sdata = sdata.astype(np.float64)
    tolerance = 0.9
    u, s,v = np.linalg.svd(sdata, full_matrices=False)
    s = np.diag(s)
    to_cut = int(tolerance*np.shape(s)[0])
    #print(np.shape(u),np.shape(s),np.shape(v))
    u = u[:,:-to_cut]
    s = s[:-to_cut,:-to_cut]
    v = v[:-to_cut,:]
    #print(np.shape(u),np.shape(s),np.shape(v))
    a = u @ s @ v
    return a

In [10]:
def recommend_me(users): #I tried to vectorize it but for some reason it changed the suggetions, so 
    #I guess I screw it up in some part of this cell
    system = recommend_system(sparse_matrix)
    scores = np.zeros((len(users),np.shape(system)[1]))
    known_taste = sparse_matrix[users,:]
    not_seen_movies = (known_taste==0) #get which movies the users have not seen
    scores = np.multiply(not_seen_movies,system[users])
    print(np.shape(scores))
    top_ind = np.argpartition(scores, -3,axis=1)[:,-3:] #get top 3 suggestions
    max_score = np.max(scores,axis=1)
    print(max_score)
    for user in np.arange(0,np.shape(top_ind)[0]): #double for just for printing
        for movie_ind in np.arange(0,np.shape(top_ind)[1]):
             movie = top_ind[user,movie_ind]
             percentage = "{0:.0%}".format(scores[user,movie]/max_score[user])
             print('User', users[user], ', we are', percentage, 'sure you will like this movie:',title_array[movie])
        print('\n')
    return None

In [11]:
recommend_me([896,1432,5329])

(3, 3706)
[2.22431257 2.50249839 1.1854576 ]
User 896 , we are 94% sure you will like this movie: Waiting to Exhale (1995)
User 896 , we are 98% sure you will like this movie: Roustabout (1964)
User 896 , we are 100% sure you will like this movie: His Girl Friday (1940)


User 1432 , we are 69% sure you will like this movie: Rock, The (1996)
User 1432 , we are 100% sure you will like this movie: Pillow Book, The (1995)
User 1432 , we are 73% sure you will like this movie: Mrs. Doubtfire (1993)


User 5329 , we are 91% sure you will like this movie: Mary Poppins (1964)
User 5329 , we are 96% sure you will like this movie: Wild Wild West (1999)
User 5329 , we are 100% sure you will like this movie: Wayne's World (1992)


